In [5]:
using Pkg
Pkg.activate("../Dashboard/env")

  Activating project at `c:\Users\Vlad\Projects\Dashboard\env`


In [22]:
Pkg.status()

Status `C:\Users\Vlad\Projects\Dashboard\env\Project.toml`
  [cbdf2221] AlgebraOfGraphics v0.6.16
  [13f3f980] CairoMakie v0.10.8
  [35d6a980] ColorSchemes v3.23.0
  [5ae59095] Colors v0.12.10
  [34da2185] Compat v4.9.0
  [8f03c58b] Effects v1.1.0
  [e9467ef8] GLMakie v0.8.8
  [5c1252a2] GeometryBasics v0.4.9
  [033835bb] JLD2 v0.4.33
  [ee78f7c6] Makie v0.19.8
  [19eb6ba3] Packing v0.5.0
  [b98c9c47] Pipe v1.3.0
  [1277b4bf] ShiftedArrays v2.0.0
⌅ [2913bbd2] StatsBase v0.33.21
  [2bdbdf9c] TopoPlots v0.1.4
⌃ [181c99d8] Unfold v0.3.4
  [69a5ce3b] UnfoldMakie v0.3.3
  [276b4fcb] WGLMakie v0.8.12
  [37e2e46d] LinearAlgebra
Info Packages marked with ⌃ and ⌅ have new versions available, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


In [2]:
Pkg.build("GLMakie")

    Building ModernGL → `C:\Users\Vlad\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\b76ea40b5c0f45790ae09492712dd326208c28b2\build.log`


Precompiling 

project...

  ✓ ModernGL


  ✗ GLMakie
  1 dependency successfully precompiled in 20 seconds. 359 already precompiled.
  1 dependency errored. To see a full report either run `import Pkg; Pkg.precompile()` or load the package


In [6]:
using Colors
using TopoPlots
using StatsBase # mean/std
using Pipe
using ColorSchemes
using LinearAlgebra
using JLD2 # loading data
using Makie
using WGLMakie


In [17]:
Makie.inline!(true)

true

In [7]:
data, pos = TopoPlots.example_data()

([0.020321019 0.017548025 … -0.01600543 -0.01283471; 0.0031529826 -0.0015369694 … 0.0435155 0.042441286; … ; 0.019118542 0.018276842 … -0.0016676243 0.000102287915; 0.019761175 0.015906857 … 0.11238504 0.10997909;;; 0.12287474 0.12434518 … 0.26227304 0.2553188; 0.07275557 0.07255436 … 0.30493587 0.30125305; … ; 0.06405158 0.06326836 … 0.2052622 0.2037698; 0.07259608 0.07544017 … 0.2627769 0.25834358;;; 0.26609147 0.2966399 … 0.40829125 0.4242154; 0.43455398 0.46788654 … 0.29459625 0.29695213; … ; 0.13350745 0.14102341 … 0.48767123 0.4992381; 0.15481107 0.21390781 … 0.059936017 0.06072311], Point{2, Float32}[[0.49371386, 0.5440313], [0.5630452, 0.50400287], [0.5630452, 0.4239459], [0.49371386, 0.38391745], [0.4243825, 0.4239459], [0.4243825, 0.50400287], [0.5378472, 0.6178857], [0.61455333, 0.56901854], [0.6522695, 0.4862579], [0.6388263, 0.39630732]  …  [0.93907887, 0.6439135], [0.9450873, 0.29968786], [0.8333667, 0.12432156], [0.61803544, 1.9428903f-16], [0.3693923, 2.7755576f-17], [0

# 0. Modular functions

In [8]:
function posToColorRGB(pos)
    cx = 0.5 - pos[1]
    cy = 0.5 - pos[2]
    # rotate to mimick MNE
    rx = cx * 0.7071068 + cy * 0.7071068
    ry = cx * -0.7071068 + cy * 0.7071068
    b = 1.0 - (2*sqrt(cx^2+cy^2))^2 # weight by distance
    colorwheel = RGB(0.5 - rx*1.414, 0.5 - ry*1.414, b)
    
    return colorwheel
end

function posToColorHSV(pos)
    rx = 0.5 - pos[1]
    ry = 0.5 - pos[2]
    
    b = 0.5#1.0 - (2*sqrt(cx^2+cy^2))^2
    θ,r =  cart2pol.(rx,ry)
   
    colorwheel = HSV(θ*360,b,(r./0.7)./2+0.5)

    return colorwheel
end


function posToColorRomaO(pos)
    rx = 0.5 - pos[1]
    ry = 0.5 - pos[2]

    θ,r =  cart2pol.(rx,ry)
    # circular colormap 2D
    colorwheel = get(ColorSchemes.romaO,θ)
    return colorwheel
end


function cart2pol(x,y)
θ = atan(x,y) ./(2*π)+0.5
r = sqrt(x^2 + y^2)
    return θ, r
end

cart2pol (generic function with 1 method)

In [19]:
function ft_topo_axis(f, sensornum, times, data;
    time=nothing, width_height = 175, labels=nothing, mark_color=nothing, markersize = 1, interpolation=ClaughTochter(), colormap = Reverse(:RdBu))

    N = 1:sensornum
    topo_axis = Axis(f, width = width_height, height = width_height, aspect = DataAspect())
    if time == nothing
        topo = eeg_topoplot!(topo_axis, mark_color, N, labels;
        positions=pos[1:sensornum], 
        enlarge=1,
        interpolation=interpolation,
        colormap = colormap,
        label_scatter=(markersize=markersize, strokewidth=0.5) 
    )
    else
        topo_slice = lift((t, data) -> mean(data[1:sensornum, indexin(t, times), :], dims=2)[:,1], time, data)
        
        topo = eeg_topoplot!(topo_axis, topo_slice, N; 
        positions=pos[1:sensornum], 
        enlarge=1,
        interpolation=interpolation,
        colormap = colormap,
        label_scatter=(markersize=markersize, strokewidth=0.5) 
        ) 
    end
end

ft_topo_axis (generic function with 1 method)

# 1. Sliders

## 1.0 topoplot

In [20]:
let 
    data, pos = TopoPlots.example_data()
    f = Figure(resolution = (1000, 900))
    xs = range(-0.3, length=size(data, 2), step=1 ./ 128) 

    topo_axis = Axis(f[1, 1], aspect = DataAspect())
    topo_slice = data[:,120,1]
    topo_axis = ft_topo_axis(f[1, 1], length(pos), xs, data; 
        time=nothing, topo_slice=topo_slice,  width_height=900, markersize = 10, label_text=false)

    f
end

MethodError: MethodError: no method matching ft_topo_axis(::GridPosition, ::Int64, ::StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, ::Array{Float32, 3}; time::Nothing, topo_slice::Vector{Float32}, width_height::Int64, markersize::Int64, label_text::Bool)

Closest candidates are:
  ft_topo_axis(::Any, ::Any, ::Any, ::Any; time, width_height, labels, mark_color, markersize, interpolation, colormap) got unsupported keyword arguments "topo_slice", "label_text"
   @ Main c:\Users\Vlad\Projects\Dashboard\prototype.ipynb:1


## 1.1. Topoplot with timeslider

In [18]:
function slider_topoplot(data, pos; 
    title="Interactive topoplot", 
    interpolation=ClaughTochter()) 
    f = Figure(resolution = (900, 600))
    N = 1:length(pos)
    xs = range(0, length=size(data, 2), step=1 ./ 100) 
    sg = SliderGrid(f[2, 1],
        (label = "time", range = xs, format = "{:.3f} ms", startvalue = 0),
    )
    time = sg.sliders[1].value

    topo_axis = ft_topo_axis(f[1, 1], length(pos), xs, data; 
    time=time, width_height=500, markersize = 1)
    # decrement/increment slider with left/right keys
    on(events(f).keyboardbutton) do btn
        if btn.action in (Keyboard.press, Keyboard.repeat)
            if btn.key == Keyboard.left
                set_close_to!(sg.sliders[1], time[] - 1)
            elseif btn.key == Keyboard.right
                set_close_to!(sg.sliders[1], time[] + 1)
            end
        end
    end
    str = lift(t -> "[$(round(t, digits = 3)) ms]", time)
    text!(topo_axis, 1, 1, text = str,  align = (:center, :center))
    
    f
end

slider_topoplot(data, pos)

UndefVarError: UndefVarError: `topo_slice` not defined

## 1.2. Combination of butterflyplot and topoplot

In [ ]:

function slider_butterfly_topoplot(data, pos; 
    title="Interactive topoplot", 
    interpolation=ClaughTochter(),
    sensornum=64, startpoint=-0.2, steps=100) 
N = 1:sensornum
onset = 0.0
steps = 1 ./ steps
times = range(startpoint, length=size(data, 2), step=steps)  
endpoint = times[end]
f = Figure(backgroundcolor = RGBf(0.98, 0.98, 0.98), resolution = (1500, 700))

# interaction

sg = SliderGrid(f[4, 1:3],
    (label="time", range=times, format = "{:.3f} ms", startvalue = 0),
)
time = sg.sliders[1].value
str = lift(t -> "$(round(t, digits = 3)) ms", time)

specialColors = ColorScheme(vcat(RGB(1,1,1.),[posToColorRomaO(pos) for pos in pos[N]]...))

# butterfly plot
ax = Axis(f[2:3, 1:3], xlabel = "Time [s]", ylabel = "Voltage amplitude [µV]")

hidespines!(ax, :t, :r) 
xlims!(startpoint, endpoint)
hlines!(0, color = :gray, linewidth = 1)
vlines!(onset, color = :gray, linewidth = 1)


datl = [mean(data[i,:,:],dims=2)[:,1] for i = 1:sensornum]
datl2 = reduce(hcat, datl)'
datl2 = datl2 .- mean(datl2[:,times .< onset], dims=2)
series!(ax, times, datl2, color = specialColors[2:sensornum+1])

hidedecorations!(ax, label = false, ticks = false, ticklabels = false) 
vlines!(time, color = :red, linewidth = 1)

ft_topo_axis(f[1, 2], sensornum, time=time, times, data, width_height=175, markersize = 1)


Label(f[1, 2], str,
    fontsize =36, font = :bold, padding = (40, 500, 0, 0), halign = :right)

hidedecorations!(current_axis())
hidespines!(current_axis())
f
end

slider_butterfly_topoplot(data, pos)


## 1.3. Buterfly plot with interactvie mode 

In [12]:

function toggle_butterfly_topoplot(data, pos; title="Interactive topoplot", interpolation=ClaughTochter(),
    sensornum=30, startpoint=-0.2, steps=100) 
    N = 1:sensornum
    onset = 0.0
    steps = 1 ./ steps
    times = range(startpoint, length=size(data, 2), step=steps)  
    endpoint = times[end]

    running = Observable(false)
    function toggle_running()
        running[] = !running[]
    end

    f = Figure(backgroundcolor = RGBf(0.98, 0.98, 0.98), resolution = (1500, 700))
    ax = Axis(f[1:3,1], xlabel = "Time [s]", ylabel = "Voltage amplitude [µV]")

    hidespines!(ax, :t, :r) 
    xlims!(startpoint, endpoint)
    hlines!(onset, color = :gray, linewidth = 1)
    vlines!(onset, color = :gray, linewidth = 1)
    
    index = Observable(2)
    multicolor = ColorScheme(vcat(RGB(1,1,1.),[posToColorRomaO(pos) for pos in pos[N]]...))[2:sensornum+1]
    solidcolor = collect(ColorScheme(range(colorant"#4d4d4d", colorant"#4d4d4d", length=sensornum)))
    specialColors = @lift($running ? multicolor : solidcolor) 

    datl = [mean(data[i,:,:], dims=2)[:,1] for i = 1:sensornum]
    datl2 = reduce(hcat, datl)'
    datl2 = datl2 .- mean(datl2[:, times .< onset], dims=2) # basline correction !!!!
    ser = series!(ax, times, datl2, color=specialColors)

    hidedecorations!(ax, label = false, ticks = false, ticklabels = false) 
    ft_topo_axis(f[1, 2], sensornum, colormap=specialColors, time=nothing, times, data, width_height=325, markersize = 20, interpolation=NullInterpolator())
  
    f[3, 2] = buttongrid = GridLayout(tellwidth = false)
    btn = Button(f, label = @lift($running ? "COLORED" : "SOLID"))
    buttongrid[1, 1:1] = [btn]

    on(btn.clicks) do _
        toggle_running()
    end
    f
end

toggle_butterfly_topoplot(data, pos)

UndefVarError: UndefVarError: `topo_slice` not defined

# 2. Clickers

## 2.1. Click topoplot

In [13]:
function click_topoplot(data, pos; interpolation=ClaughTochter(),
    sensornum=64) 

    f = Figure(backgroundcolor = RGBf(0.98, 0.98, 0.98))
    N = 1:length(pos)

    topo_axis = Axis(f[1, 1],  aspect = DataAspect())
    
	xlims!(low = -0.2, high = 1.2)
	ylims!(low = -0.2, high = 1.2)
    
    mark_size = repeat([21], sensornum)
    mark_color = repeat([1], sensornum)
    labels = ["s$i" for i in 1:sensornum]
    topo = eeg_topoplot!(topo_axis, data[:, 340, 1], labels;
        mark_color,  N, 
        positions=pos[1:sensornum], 
        interpolation=NullInterpolator(),
        enlarge=1,
        label_text=false, 
        label_scatter=(markersize=mark_size, color=:black)
    ) 
    hidedecorations!(current_axis())
    hidespines!(current_axis())

    i = Observable(1)
    str = lift((i, labels) -> "$(labels[i])", i, labels)
    text!(topo_axis, 1, 1, text = str,  align = (:center, :center))
    on(events(f).mousebutton, priority = 2) do event
        if event.button == Mouse.left && event.action == Mouse.press
            plt, p = pick(topo_axis)
            i[] = p
        end
    end
    f
end

click_topoplot(data, pos)

┌ Warning: Port in use, using different port. New port: 9385
└ @ JSServe.HTTPServer C:\Users\Vlad\.julia\packages\JSServe\qgqel\src\HTTPServer\implementation.jl:273


## 2.2. Click butterfly

In [16]:
function click_butterfly_topoplot(data, pos; title="Interactive topoplot", interpolation=ClaughTochter(),
    sensornum=64, startpoint=-0.2, steps=100)  

    N = 1:sensornum
    onset = 0.0
    steps = 1 ./ steps
    times = range(startpoint, length=size(data, 2), step=steps)  
    endpoint = times[end]

    f = Figure(#backgroundcolor = RGBf(0.98, 0.98, 0.98), 
    resolution = (1500, 700))
    ax = Axis(f[1:3, 1], xlabel = "Time [s]", ylabel = "Voltage amplitude [µV]")

    hidespines!(ax, :t, :r) 
    xlims!(startpoint, endpoint)
    hlines!(0, color = :gray, linewidth = 1)
    vlines!(0, color = :gray, linewidth = 1)
    
    datl = [mean(data[i,:,:], dims=2)[:,1] for i = 1:sensornum]
    datl2 = reduce(hcat, datl)'
    datl2 = datl2 .- mean(datl2[:,times .< onset], dims=2) # basline correction

    i = Observable(15)
    begin
        tmp1 = repeat([0], sensornum)
        mark_color = @lift begin
            tmp1[1:end] .= 0
            if $i != 0
                tmp1[$i] = 1
            end
            return tmp1

        end
        red_line = @lift begin
            datl3 = datl2'
            tmp2 = datl3[:, 1]  
                if $i != 0
                    tmp2 = datl3[:, $i]
                end
                return tmp2
        end
    end

    series!(ax, times, datl2, solid_color="#4d4d4d")

    cmap = collect(ColorScheme(range(colorant"black", colorant"red", length=2)))
    mark_size = repeat([21], sensornum)
    topo_axis = ft_topo_axis(f[2:3, 2], sensornum, times, data; mark_color=mark_color, colormap = cmap, 
    interpolation=NullInterpolator(), time=nothing, width_height=345, 
    markersize = mark_size)
    
    labels = ["s$i" for i in 1:sensornum]
    str = lift((i, labels) -> "$(labels[i])", i, labels)
    Label(f[1, 2], str,
        fontsize =36, font = :bold, padding = (0, 0, 0, 50), halign = :center)

    lc = nothing
    on(events(f).mousebutton, priority = 2) do event
        if event.button == Mouse.left && event.action == Mouse.press

            plt, p = pick(topo_axis)
            i[] = p
            if lc != nothing
                lc.color  = "#4d4d4d"
                lc.linewidth  = 1.5
            end
            lc = lines!(ax, times, red_line[], color = "red", linewidth = 5)

        end
    end
    hidedecorations!(ax, label = false, ticks = false, ticklabels = false) 
    f
end

click_butterfly_topoplot(data, pos)


MethodError: MethodError: no method matching length(::Nothing)

Closest candidates are:
  length(!Matched::Union{Base.KeySet, Base.ValueIterator})
   @ Base abstractdict.jl:58
  length(!Matched::Union{SparseArrays.FixedSparseVector{Tv, Ti}, SparseArrays.SparseVector{Tv, Ti}} where {Tv, Ti})
   @ SparseArrays C:\Users\Vlad\AppData\Local\Programs\julia-1.9.2\share\julia\stdlib\v1.9\SparseArrays\src\sparsevector.jl:95
  length(!Matched::Union{Adjoint{T, <:Union{StaticArraysCore.StaticArray{Tuple{var"#s2"}, T, 1} where var"#s2", StaticArraysCore.StaticArray{Tuple{var"#s3", var"#s4"}, T, 2} where {var"#s3", var"#s4"}}}, Diagonal{T, <:StaticArraysCore.StaticArray{Tuple{var"#s14"}, T, 1} where var"#s14"}, Hermitian{T, <:StaticArraysCore.StaticArray{Tuple{var"#s11", var"#s12"}, T, 2} where {var"#s11", var"#s12"}}, LowerTriangular{T, <:StaticArraysCore.StaticArray{Tuple{var"#s19", var"#s20"}, T, 2} where {var"#s19", var"#s20"}}, Symmetric{T, <:StaticArraysCore.StaticArray{Tuple{var"#s8", var"#s

# 2.3. Menu

In [24]:
function menu_butterfly_topoplot(data, pos; title="Interactive topoplot", interpolation=ClaughTochter(),
    sensornum=64, startpoint=-0.2, steps=100) 
    N = 1:sensornum
    onset = 0.0
    steps = 1 ./ steps
    times = range(startpoint, length=size(data, 2), step=steps)  
    endpoint = times[end]
    
    f = Figure(backgroundcolor = RGBf(0.98, 0.98, 0.98), resolution = (1500, 700))
    ax = Axis(f[1:3, 1], xlabel = "Time [s]", ylabel = "Voltage amplitude [µV]")

    N = 1:length(pos)
    hidespines!(ax, :t, :r) 
    xlims!(startpoint, endpoint)
    hlines!(0, color = :gray, linewidth = 1)
    vlines!(0, color = :gray, linewidth = 1)
    
    datl = [mean(data[i,:,:],dims=2)[:,1] for i = 1:sensornum]
    datl2 = reduce(hcat, datl)'
    datl2 = datl2 .- mean(datl2[:,times .< onset], dims=2) # basline correction

    num = Observable(0)

    begin
        tmp1 = repeat([0], sensornum)
        mark_color = @lift begin
            tmp1[1:end] .= 0
            if $num != 0
                tmp1[$num] = 1
            end
            return tmp1
        end
        red_line = @lift begin
            datl3 = datl2'
            tmp2 = datl3[:, 1]  
                if $num != 0
                    tmp2 = datl3[:, $num]
                end
                return tmp2
        end
    end

    series!(ax, times, datl2, solid_color="#4d4d4d")

    hidedecorations!(ax, label = false, ticks = false, ticklabels = false) 
    cmap = collect(ColorScheme(range(colorant"black", colorant"red", length=2)))
    
    mark_size = repeat([21], sensornum)
    labels = ["s$i" for i in 1:sensornum]
    menu = Menu(f[3, 2], options = labels, default = nothing)

    topo_axis = ft_topo_axis(f[1:2, 2], sensornum, mark_color=mark_color, labels=labels, colormap = cmap, 
        interpolation=NullInterpolator(), time=nothing, times, data, width_height=345, markersize = mark_size)

    lc = nothing
        on(menu.selection) do selected
        if selected != nothing
            if lc != nothing
                lc.color  = "#4d4d4d"
                lc.linewidth  = 1.5
            end
            num[] = findfirst(x->x==menu.selection[], labels)
            lc = lines!(ax, times, red_line[], color = "red", linewidth = 5)
            notify(mark_color)
            notify(red_line)   
        end
    end
    notify(menu.selection) 
    hidedecorations!(topo_axis)
    hidespines!(topo_axis)
    f

end

menu_butterfly_topoplot(data, pos)

# Combination of butterflyplot and topoplot series
- butterflyplot
- topoplot series underneath 
- you can change number of topoplots and their range